In [4]:
from PIL import Image
from math import floor
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from pynq import allocate, Overlay
from random import seed
from random import randint
from random import getrandbits
from pprint import pprint

In [5]:
def run_kernel():
    dma.sendchannel.transfer(in_buffer)
    dma.recvchannel.transfer(out_buffer)
    dma.recvchannel.wait()
    dma.sendchannel.wait()

In [6]:
filterBase = 24
dataBase = 60
def output_registers():
    print("Control Enable:         %d" % (controller.read(0)))
    print("Reset:                  %d" % (controller.read(4)))
    print("State Machine Register: %d" % (controller.read(8)))
    print("Last Calculated Value:  %d" % (controller.read(12)))
    print("Image Width:            %d" % (controller.read(16)))
    print("Image Height:           %d" % (controller.read(20)))
    print("\nFilter Set:")
    for i in range(9):
        print("filterSet[%d] = %d" % (i,controller.read((i*4)+filterBase)))
    print("\nData Set:")
    for i in range(9):
        print("dataSet[%d] = %d" % (i,controller.read((i*4)+dataBase)))
    print("\n")

In [7]:
bit_precision = 0

if(bit_precision==0):
    Convolution_design = Overlay("Hardware-Files/Stable/PL_Convolution_8b.bit",)
elif(bit_precision==1):
    Convolution_design = Overlay("Hardware-Files/Stable/PL_Convolution_16b.bit",)
elif(bit_precision==2):
    Convolution_design = Overlay("Hardware-Files/Stable/PL_Convolution_32b.bit",)

dma = Convolution_design.axi_dma_0
controller = Convolution_design.Convolution_Controll_0

In [8]:
width = 16
height = 16
kernel_size = 3

# Creating Correct precision input buffer
if(bit_precision==0):
    in_buffer = allocate(shape=(height, width, 1),dtype=np.uint8, cacheable=1)
elif(bit_precision==1):
    in_buffer = allocate(shape=(height, width, 1),dtype=np.uint16, cacheable=1)
elif(bit_precision==2):
    in_buffer = allocate(shape=(height, width, 1),dtype=np.uint32, cacheable=1)    
out_buffer = allocate(shape=(height-2, width-2, 1),dtype=np.uint32, cacheable=1)

for i in range(width):
    for j in range(height):
        in_buffer[i][j] = i*width+j
#         in_buffer[j][i] = 10 if bool(getrandbits(1)) else 100

In [38]:
# in_buffer

In [10]:
kernelsize = 3
# High-pass kernel
kernel = [[ 1, 1, 1 ],
          [ 1, 1, 1 ],
          [ 1, 1, 1 ]]

In [11]:
#Enable IP control register
controller.write(0,1)

#Input Filter set
for x in range(kernelsize):
    for y in range(kernelsize):
        controller.write(filterBase+((x*kernelsize)+y)*4,kernel[y][x])

#Input width/height
controller.write(16,width)#Width
controller.write(20,height)#Height

In [12]:
output_registers()

Control Enable:         1
Reset:                  0
State Machine Register: 1
Last Calculated Value:  0
Image Width:            16
Image Height:           16

Filter Set:
filterSet[0] = 1
filterSet[1] = 1
filterSet[2] = 1
filterSet[3] = 1
filterSet[4] = 1
filterSet[5] = 1
filterSet[6] = 1
filterSet[7] = 1
filterSet[8] = 1

Data Set:
dataSet[0] = 0
dataSet[1] = 0
dataSet[2] = 0
dataSet[3] = 0
dataSet[4] = 0
dataSet[5] = 0
dataSet[6] = 0
dataSet[7] = 0
dataSet[8] = 0




In [13]:
%%timeit
run_kernel()

The slowest run took 4.58 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 837 µs per loop


In [14]:
output_registers()

Control Enable:         1
Reset:                  0
State Machine Register: 1
Last Calculated Value:  0
Image Width:            16
Image Height:           16

Filter Set:
filterSet[0] = 1
filterSet[1] = 1
filterSet[2] = 1
filterSet[3] = 1
filterSet[4] = 1
filterSet[5] = 1
filterSet[6] = 1
filterSet[7] = 1
filterSet[8] = 1

Data Set:
dataSet[0] = 0
dataSet[1] = 0
dataSet[2] = 0
dataSet[3] = 0
dataSet[4] = 0
dataSet[5] = 0
dataSet[6] = 0
dataSet[7] = 0
dataSet[8] = 0




In [15]:
for i in range(height):
    for j in range(width):
        print(hex(in_buffer[i,j,0]),end=' ')
        if(j==width-1):
            print()

0x0 0x1 0x2 0x3 0x4 0x5 0x6 0x7 0x8 0x9 0xa 0xb 0xc 0xd 0xe 0xf 
0x10 0x11 0x12 0x13 0x14 0x15 0x16 0x17 0x18 0x19 0x1a 0x1b 0x1c 0x1d 0x1e 0x1f 
0x20 0x21 0x22 0x23 0x24 0x25 0x26 0x27 0x28 0x29 0x2a 0x2b 0x2c 0x2d 0x2e 0x2f 
0x30 0x31 0x32 0x33 0x34 0x35 0x36 0x37 0x38 0x39 0x3a 0x3b 0x3c 0x3d 0x3e 0x3f 
0x40 0x41 0x42 0x43 0x44 0x45 0x46 0x47 0x48 0x49 0x4a 0x4b 0x4c 0x4d 0x4e 0x4f 
0x50 0x51 0x52 0x53 0x54 0x55 0x56 0x57 0x58 0x59 0x5a 0x5b 0x5c 0x5d 0x5e 0x5f 
0x60 0x61 0x62 0x63 0x64 0x65 0x66 0x67 0x68 0x69 0x6a 0x6b 0x6c 0x6d 0x6e 0x6f 
0x70 0x71 0x72 0x73 0x74 0x75 0x76 0x77 0x78 0x79 0x7a 0x7b 0x7c 0x7d 0x7e 0x7f 
0x80 0x81 0x82 0x83 0x84 0x85 0x86 0x87 0x88 0x89 0x8a 0x8b 0x8c 0x8d 0x8e 0x8f 
0x90 0x91 0x92 0x93 0x94 0x95 0x96 0x97 0x98 0x99 0x9a 0x9b 0x9c 0x9d 0x9e 0x9f 
0xa0 0xa1 0xa2 0xa3 0xa4 0xa5 0xa6 0xa7 0xa8 0xa9 0xaa 0xab 0xac 0xad 0xae 0xaf 
0xb0 0xb1 0xb2 0xb3 0xb4 0xb5 0xb6 0xb7 0xb8 0xb9 0xba 0xbb 0xbc 0xbd 0xbe 0xbf 
0xc0 0xc1 0xc2 0xc3 0xc4 0xc5 0xc6 0xc7 0xc8

In [16]:
for i in range(height-2):
    for j in range(width-2):
        print(hex(out_buffer[i,j,0]),end=' ')
        if(j==width-3):
            print()

0x0 0x33 0x36 0x39 0x3c 0x3f 0x42 0x45 0x48 0x4b 0x4e 0x51 0x54 0x57 
0x0 0x63 0x66 0x69 0x6c 0x6f 0x72 0x75 0x78 0x7b 0x7e 0x81 0x84 0x87 
0x0 0x83 0x86 0x89 0x8c 0x8f 0x92 0x95 0x98 0x9b 0x9e 0xa1 0xa4 0xa7 
0x0 0xc3 0xc6 0xc9 0xcc 0xcf 0xd2 0xd5 0xd8 0xdb 0xde 0xe1 0xe4 0xe7 
0x0 0xf3 0xf6 0xf9 0xfc 0xff 0x2 0x5 0x8 0xb 0xe 0x11 0x14 0x17 
0x0 0x13 0x16 0x19 0x1c 0x1f 0x22 0x25 0x28 0x2b 0x2e 0x31 0x34 0x37 
0x0 0x53 0x56 0x59 0x5c 0x5f 0x62 0x65 0x68 0x6b 0x6e 0x71 0x74 0x77 
0x0 0x83 0x86 0x89 0x8c 0x8f 0x92 0x95 0x98 0x9b 0x9e 0xa1 0xa4 0xa7 
0x0 0xa3 0xa6 0xa9 0xac 0xaf 0xb2 0xb5 0xb8 0xbb 0xbe 0xc1 0xc4 0xc7 
0x0 0xe3 0xe6 0xe9 0xec 0xef 0xf2 0xf5 0xf8 0xfb 0xfe 0x1 0x4 0x7 
0x0 0x13 0x16 0x19 0x1c 0x1f 0x22 0x25 0x28 0x2b 0x2e 0x31 0x34 0x37 
0x0 0x33 0x36 0x39 0x3c 0x3f 0x42 0x45 0x48 0x4b 0x4e 0x51 0x54 0x57 
0x0 0x73 0x76 0x79 0x7c 0x7f 0x82 0x85 0x88 0x8b 0x8e 0x91 0x94 0x97 
0x0 0xa3 0xa6 0xa9 0xac 0xaf 0xb2 0xb5 0xb8 0xbb 0xbe 0xc1 0xc4 0xc7 


In [2]:
filterSet = [ -1,-2,-1,0,0,0,1,2,1 ]
dataSet = [ 230,229,230,232,231,230,231,231,229 ]
cSum = 0

for i in range(9):
    if (i%3==0):
        print()
        
    print("%s x %s = %s = %d" % (hex(dataSet[i]),hex(filterSet[i]),hex(dataSet[i]*filterSet[i]),dataSet[i]*filterSet[i]))
    cSum+=dataSet[i]*filterSet[i]

print("\ncSum in hex:     %s" % (hex(cSum)))
print("cSum in decimal: %d" % (cSum))


0xe6 x -0x1 = -0xe6 = -230
0xe5 x -0x2 = -0x1ca = -458
0xe6 x -0x1 = -0xe6 = -230

0xe8 x 0x0 = 0x0 = 0
0xe7 x 0x0 = 0x0 = 0
0xe6 x 0x0 = 0x0 = 0

0xe7 x 0x1 = 0xe7 = 231
0xe7 x 0x2 = 0x1ce = 462
0xe5 x 0x1 = 0xe5 = 229

cSum in hex:     0x4
cSum in decimal: 4


In [21]:
out_buffer

PynqBuffer([[[  0],
             [  0],
             [  1],
             [  3],
             [  4],
             [  5],
             [  6],
             [  7],
             [  8],
             [  9],
             [ 10],
             [ 11],
             [ 12],
             [ 13]],

            [[  0],
             [ 16],
             [ 17],
             [ 19],
             [ 20],
             [ 21],
             [ 22],
             [ 23],
             [ 24],
             [ 25],
             [ 26],
             [ 27],
             [ 28],
             [ 29]],

            [[  0],
             [ 32],
             [ 33],
             [ 35],
             [ 36],
             [ 37],
             [ 38],
             [ 39],
             [ 40],
             [ 41],
             [ 42],
             [ 43],
             [ 44],
             [ 45]],

            [[  0],
             [ 48],
             [ 49],
             [ 51],
             [ 52],
             [ 53],
             [ 54],
             [